# Setup

## Check GPU
In order to perform the experiments in a reasonable time, check whether the GPU has at least 15GiB. If it is not, it's necessary to restart the runtime until this requirement is satisfied

In [ ]:
!nvidia-smi

## Installations and imports


### Installations
As some libraries that are not in the default version in colab are used, it is necessary to install them

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-geometric -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html

!pip install torchmetrics

### Imports
In the next snippet of code there are all the imports necessaries for the project and the tensorboard is initialized.

In [ ]:
import os
import datetime
import numpy as np

import tensorflow
import tensorboard
import plotly.graph_objects as go

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter

from torchmetrics import Accuracy

from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_batch
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints, NormalizeScale, RandomFlip, RandomRotate, Compose

%reload_ext tensorboard

## Hyperparameters

In [ ]:
hparams = {
    'bs': 32,
    'epochs': 100,
    'device': torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    'tb_logs': '/content/drive/MyDrive/Adapt',
    'tb_name': 'tb_' + str(datetime.datetime.utcnow()),
    'drive_root': '/content/drive/MyDrive/Dataset/',
    'normalize_scale': True, 
    'data_augmentation': 'flip_rotate',
    'fixed_num_of_points': 1024,
    'flip_probability': 0.5,
    'flip_axis': 1,
    'rotate_degrees': 45,
    'rotate_axis': 0,
    'model_log': '/content/drive/MyDrive/Adapt/', 
    'model_name': 'pointNet_flip_rotate_0.3_adam_OneCycleLR',
    'k': 3,
    'num_classes': 10,
    'dropout': 0.3,
    'optimizer': 'Adam',
    'lr': 1e-3,
    'wd': 1e-3,
    'momentum': 0.9,
    'scheduler': 'OneCycleLR',
    'gamma': 0.5,
    'patience': 10,
    'step_size': 20
}

## Seeds 

In [ ]:
seed = 42
# Controlling sources of randomness
torch.manual_seed(seed) # generate random numbers for all devices (both CPU and CUDA)
# Random number generators in other libraries:
np.random.seed(seed)
# CUDA convolution benchmarking:
torch.backends.cudnn.benchmark = False # ensures that CUDA selects the same algorithm each time an application is run

# Model - PointNet

In [ ]:
class TNet(nn.Module):
  def __init__(self, k=3):
    super().__init__()
    self.k = k
    
    self.Conv1 = nn.Conv1d(in_channels=k, out_channels=64, kernel_size=1)
    self.bn1 = nn.BatchNorm1d(64) 
    self.Conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
    self.bn2 = nn.BatchNorm1d(128)
    self.Conv3 = nn.Conv1d(in_channels=128, out_channels=1024, kernel_size=1)
    self.bn3 = nn.BatchNorm1d(1024)

    self.FC1 = nn.Linear(in_features=1024, out_features=512)
    self.bn4 = nn.BatchNorm1d(512)
    self.FC2 = nn.Linear(in_features=512, out_features=256)
    self.bn5 = nn.BatchNorm1d(256)

    self.FC3 = nn.Linear(in_features=256, out_features=k*k)

  def forward(self, cloud_points):
    bs = cloud_points.size(0)

    x = F.relu(self.bn1(self.Conv1(cloud_points)))
    x = F.relu(self.bn2(self.Conv2(x)))
    x = F.relu(self.bn3(self.Conv3(x)))

    # size: [batch size, 1024, # of points]
    x = nn.MaxPool1d(x.size(-1))(x) # pool with kernel = # of points/batch
    # size: [batch size, 1024, 1]
    x = x.view(bs,-1) # flatten to get horizontal vector

    # size: [batch size, 1024]
    x = F.relu(self.bn4(self.FC1(x)))
    x = F.relu(self.bn5(self.FC2(x)))

    # diagonal matrices initialized, as many as batch size
    init_matrix = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
    if x.is_cuda:
      init_matrix = init_matrix.cuda() # gets updated according to f.c. output
    matrix = self.FC3(x).view(-1, self.k, self.k) + init_matrix
    return matrix

class Transform(nn.Module):
   def __init__(self, k=3):
        super().__init__()
        self.input_transform = TNet(k)
        self.feature_transform = TNet(k=64)

        self.Conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=1)
        self.Conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
        self.Conv3 = nn.Conv1d(in_channels=128, out_channels=1024, kernel_size=1)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

   def forward(self, x):
        bs = x.size(0)

        matrix3x3 = self.input_transform(x)
        x = torch.bmm(torch.transpose(x,1,2),matrix3x3).transpose(1,2)
        x = F.relu(self.bn1(self.Conv1(x)))
        
        matrix64x64 = self.feature_transform(x)
        x = torch.bmm(torch.transpose(x,1,2), matrix64x64).transpose(1,2) 
        x = F.relu(self.bn2(self.Conv2(x)))
        x = self.bn3(self.Conv3(x))

        x = nn.MaxPool1d(x.size(-1))(x)
        global_features = x.view(bs,-1)
        return global_features

class PointNetModel(nn.Module):
    def __init__(self, k=3, num_classes=16, dropout=0.3):
        super().__init__()
        self.transform = Transform(k)

        self.FC1 = nn.Linear(in_features=1024, out_features=512)
        self.bn1 = nn.BatchNorm1d(512)
        self.FC2 = nn.Linear(in_features=512, out_features=256)
        self.bn2 = nn.BatchNorm1d(256)
        self.FC3 = nn.Linear(in_features=256, out_features=num_classes)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        global_features = self.transform(x)
        x = F.relu(self.bn1(self.FC1(global_features)))
        x = self.FC2(x)
        # apply dropout if exists
        x = self.dropout(x) if self.dropout is not None else x
        x = F.relu(self.bn2(x))
        output = self.FC3(x)
        return output, F.softmax(output,dim=1)

# Dataset
First of all, it is necessary to make the drive folder with the dataset available to this collab in order not to download it every time. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Transformations
In this project is necessary to use some transformations to either normalize the data or to perform data augmentation.

In [ ]:
def get_pre_transformation(number_points=1024):
    return SamplePoints(num=number_points)

def get_transformation(normalize_scale):
    if normalize_scale:  
      return NormalizeScale()
    else:
        return None

def get_random_flip(axis=1, p=0.5):
    return RandomFlip(axis, p)


def get_random_rotation(degrees=45, axis=1):
    return RandomRotate(degrees, axis)


def data_augmentation_flip(normalize_scale, axis=1, p=0.5):
    return Compose([get_transformation(normalize_scale), get_random_flip(axis, p)])


def data_augmentation_rotation(normalize_scale, axis=1, degrees=45):
    return Compose([get_transformation(normalize_scale), get_random_rotation(axis=axis, degrees=degrees)])


def data_augmentation_flip_rotation(normalize_scale, axis_flip=1, p=0.5, axis_rotation=1, degrees=45):
    return Compose([get_transformation(normalize_scale), get_random_flip(axis_flip, p),
                    get_random_rotation(axis=axis_rotation, degrees=degrees)])


def get_data_augmentation(dataset, transformation, normalize_scale, axis_flip=1, p=0.5, axis_rotation=1, degrees=45):
  if transformation is not None:
    if transformation.lower() == 'flip_rotation':
        dataset.transform = data_augmentation_flip_rotation(normalize_scale, axis_flip, p, axis_rotation, degrees)
    elif transformation.lower() == 'flip':
        dataset.transform = data_augmentation_flip(normalize_scale, axis=axis_flip, p=p)
    elif transformation.lower() == 'rotate':
        dataset.transform = data_augmentation_rotation(normalize_scale, axis=axis_rotation, degrees=degrees)


## Training, validation and test data

In this step, some processing of the data is going to be used to be able to feed it to the model. 

In [ ]:
def get_dataset(root, transform, pre_transform):
    train_valid_dataset = ModelNet(root=root, name="10", train=True, pre_transform=pre_transform, transform=transform)
    test_dataset = ModelNet(root=root, name="10", train=False, pre_transform=pre_transform, transform=transform)
    return train_valid_dataset, test_dataset

# Function used to split between validation and training
def get_split(index_file_root, dataset):
    index_file = open(index_file_root, 'r')
    train_index = []
    for idx in index_file:
        train_index.append(int(idx))

    return dataset[train_index]

# Function to decide which points are used in validation and which ones in training
def create_file_if_necessary(train_file, valid_file, dataset):
    if not os.path.isfile(train_file) and not os.path.isfile(valid_file):
        torch.manual_seed(0)
        # Shuffle before splitting data (random split)
        _, perm = dataset.shuffle(return_perm=True)

        # Create two files with the indices od the training and validation data
        train_idx = open(train_file, 'w+')
        valid_idx = open(valid_file, 'w+')

        # Split the tensor of indices in training and validation
        train_split, val_split = perm.split(round(len(perm) * 0.8))

        for i in range(len(train_split)):
            train_idx.writelines(str(train_split[i].item()) + "\n")
        for i in range(len(val_split)):
            valid_idx.writelines(str(val_split[i].item()) + "\n")
        
        print("New split file has been created")

        train_idx.close()
        valid_idx.close()

    elif not os.path.isfile(train_file) or not os.path.isfile(valid_file):
        raise ValueError('One file exists and the other one does not')

# Function to be called when creating dataset 
def get_train_valid_test_ModelNet(root, number_points=1024, normalize_scale=True):
    dataset_root = os.path.join(root, 'ModelNet')
    train_valid_split, test_split = get_dataset(dataset_root, transform=NormalizeScale() if normalize_scale else None,
                                                pre_transform=SamplePoints(num=number_points))

    train_split_root = os.path.join(root, 'train_split.txt')
    valid_split_root = os.path.join(root, 'val_split.txt')
    create_file_if_necessary(train_split_root, valid_split_root, train_valid_split)

    train_split = get_split(index_file_root=train_split_root, dataset=train_valid_split)
    valid_split = get_split(index_file_root=valid_split_root, dataset=train_valid_split)
    return train_split, valid_split, test_split

# Helper functions

As there are some functionalities that are used by different functions or can be used in the future, a list of helpers fucntions has been created

In [ ]:
# Initialize the tensorflow
def get_tensorboard_writer(root):
    tensorflow.io.gfile = tensorboard.compat.tensorflow_stub.io.gfile  # avoid tensorboard crash when adding embeddings
    train_log_dir = os.path.join(root, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), 'train')
    valid_log_dir = os.path.join(root, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), 'valid')
    train_writer = SummaryWriter(log_dir=train_log_dir)
    valid_writer = SummaryWriter(log_dir=valid_log_dir)
    return train_writer, valid_writer

# Writes information of every epoch in the tensorboard
def write_epoch_data(train_writer, valid_writer, train_loss, valid_loss, train_accuracy, valid_accuracy, epoch):
    # Write Loss and Accuracy in tensorboard:
    train_writer.add_scalar('Loss', train_loss, epoch)
    train_writer.add_scalar('Accu', train_accuracy, epoch)
    valid_writer.add_scalar('Loss', valid_loss, epoch)
    valid_writer.add_scalar('Accu', valid_accuracy, epoch)

# Funtion to save the best model so far
def update_best_model(valid_accuracy, model_state_dict, model_root, model_name):
    model_path = os.path.join(model_root, model_name + datetime.datetime.now().strftime("%Y%m%d%h"))
    torch.save(model_state_dict, model_path + '.pt')
    return valid_accuracy, model_path

# Method to visualize a cloud point
def visualize_point_cloud(point_cloud):
    points, y = point_cloud
    fig = go.Figure(data=[go.Mesh3d(x=points[1][:, 0], y=points[1][:, 1], z=points[1][:, 2], mode='markers', marker=dict(size=3, opacity=1))])
    fig.show()

# Training and testing functions

## Correct parameters
As different experiments will be performed some variables need to be dependent on hyperparameters. That is why some functions have been used due to not modify the code when performing different experiments. Adding new conditions to the if - else statement, more schedulers or optimizers can be added.

In [ ]:
def get_optimizer(optimizer_name, model_parameters, lr, wd, momentum):
    if optimizer_name.lower() == "Adam".lower():
        return torch.optim.Adam(model_parameters, lr=lr, weight_decay=wd)
    elif optimizer_name.upper() == "SGD":
        return torch.optim.SGD(model_parameters, lr=lr, momentum=momentum)
    else:
        raise ValueError('Optimizer is not correctly introduced')

def get_scheduler(scheduler_name, optimizer, lr=1e-3, gamma=0.5, patience=10, step_size=20, train_loader_len=1024,
                  num_epochs=100):
  if scheduler_name is not None:
    if scheduler_name.lower() == 'StepLR'.lower():
        return torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma), None, False
    elif scheduler_name.lower() == 'ReduceLROnPlateau'.lower():
        return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=gamma, patience=patience), None, True
    elif scheduler_name.lower() == 'OneCycleLR'.lower():
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, lr, steps_per_epoch=train_loader_len,
                                                        epochs=num_epochs)
        return scheduler, scheduler, None
    else:
      raise ValueError('Incorrect scheduler')
  else:
    print('No scheduler')
    return None, None, None

## Training loop

In [ ]:
# Train One Epoch:
def train_epoch(model, train_loader, optimizer, criterion, accuracy, device, scheduler):
    # Model in train mode:
    model.train()
    # List for epoch loss:
    epoch_train_loss = []
    # Metric stored information reset:
    accuracy.reset()

    # Train epoch loop:
    for i, data in enumerate(train_loader, 1):
        # Data retrieval from each bath:
        points = to_dense_batch(data.pos, batch=data.batch)[0].to(device).float().transpose(1, 2)
        targets = data.y.to(device)

        # Forward pass:
        preds, probs = model(points)
        # Loss calculation + Backpropagation pass
        optimizer.zero_grad()
        loss = criterion(preds.to(device), targets)
        epoch_train_loss.append(loss.item())
        loss.backward()
        optimizer.step()

        # Step for OneCycle scheduler
        if scheduler is not None:
            scheduler.step()

        # Batch metrics calculation:
        accuracy.update(probs, targets)

    # Mean epoch metrics calculation:
    mean_loss = np.mean(epoch_train_loss)
    mean_accu = accuracy.compute().item()
    # Print of all metrics:
    print('Train loss: ', mean_loss, "| Acc.: " , mean_accu)
    return mean_loss, mean_accu


# Valid One Epoch:
def valid_epoch(model, valid_loader, criterion, accuracy, device):
    # Model in validation (evaluation) mode:
    model.eval()
    # List for epoch loss:
    epoch_valid_loss = []
    # Metric stored information reset:
    accuracy.reset()
    # Batch loop for validation:
    with torch.no_grad():
        for i, data in enumerate(valid_loader, 1):
            # Data retrieval from each bath:
            points = to_dense_batch(data.pos, batch=data.batch)[0].to(device).float().transpose(1, 2)
            targets = data.y.to(device)

            # Forward pass:
            preds, probs = model(points)
            # Loss calculation
            loss = criterion(preds.to(device), targets)
            epoch_valid_loss.append(loss.item())
            # Batch metrics calculation:
            accuracy.update(probs, targets)
    # Mean epoch metrics calculation:
    mean_loss = np.mean(epoch_valid_loss)
    mean_accu = accuracy.compute().item()
    # Print of all metrics:
    print('Valid loss: ', mean_loss, "| Acc.: ", mean_accu)
    return mean_loss, mean_accu


def fit(train_data, valid_data, num_classes, k=3, bs=32, num_epochs=100, lr=1e-3):
    # Data Loaders for train and validation:
    train_loader = DataLoader(train_data, batch_size=bs, shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=bs, shuffle=False)

    # Obtain correct model
    model = PointNetModel(k, num_classes, hparams['dropout']).to(hparams['device'])

    optimizer = get_optimizer(hparams['optimizer'], model.parameters(), lr, hparams['wd'], hparams['momentum'])

    # Obtain correct scheduler: train scheduler is used to determine wheteher or not there is a scheduler in the train fucntion,
    # the fit scheduler can have three values: None (no scheduler in fit function), True (needs the valid loss parameter), 
    # False(no need of extra parameter)
    scheduler, train_scheduler, fit_scheduler = get_scheduler(hparams['scheduler'], optimizer, lr, hparams['gamma'],
                                                              hparams['patience'], hparams['step_size'],
                                                              len(train_loader),
                                                              num_epochs)
    criterion = nn.CrossEntropyLoss().to(hparams['device'])

    # Metric
    accuracy = Accuracy(average='micro', compute_on_step=False).to(hparams['device'])

    # Tensorboard set up
    train_writer, valid_writer = get_tensorboard_writer(hparams['tb_logs'])

    # Minimum accuracy to save the model
    best_accuracy = 0.0
    model_root = None
    print('Start training...')
    for epoch in range(1, num_epochs + 1):
        print('Epoch: ', epoch)
        # Train and validation
        train_loss, train_accu = train_epoch(model, train_loader, optimizer, criterion, accuracy, hparams['device'],
                                             train_scheduler)
        valid_loss, valid_accu = valid_epoch(model, valid_loader, criterion, accuracy, hparams['device'])

        # Choose scheduler
        if fit_scheduler is not None:
          if fit_scheduler:
            scheduler.step(valid_loss) 
          else: 
            scheduler.step()

        write_epoch_data(train_writer, valid_writer, train_loss, valid_loss, train_accu, valid_accu, epoch)

        # Save best model:
        if best_accuracy < valid_accu:
            best_accuracy, model_root = update_best_model(valid_accu, model.state_dict(), hparams['model_log'], hparams['model_name'])

    final_state_dict_root = model_root + '.pt'
    model.load_state_dict(torch.load(final_state_dict_root))
    print("Best val accuracy: ", best_accuracy)
    return best_accuracy, final_state_dict_root

## Test inference

In [ ]:
def test(test_data, model_state_dict_root):
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)
    model = PointNetModel(hparams['k'], hparams['num_classes'], hparams['dropout']).to(hparams['device'])

    accuracy = Accuracy(average='micro', compute_on_step=False).to(hparams['device'])
    model.load_state_dict(torch.load(model_state_dict_root))
    model.eval()
    # Metric stored information reset:
    accuracy.reset()
    # Batch loop for validation:
    with torch.no_grad():
        for i, data in enumerate(test_loader, 1):
            # Data retrieval from each bath:
            points = to_dense_batch(data.pos, batch=data.batch)[0].to(hparams['device']).float().transpose(1, 2)
            targets = data.y.to(hparams['device'])

            # Forward pass:
            preds, probs = model(points)

            # Batch metrics calculation:
            accuracy.update(probs, targets)

    mean_accu = accuracy.compute().item()
    # Print of all metrics:
    print("Test Acc.: ", mean_accu)
    return mean_accu

# Experiments
More info in the README.

In [ ]:
def execute_experiment():
  train_dataset, valid_dataset, test_dataset = get_train_valid_test_ModelNet(hparams['drive_root'], hparams['fixed_num_of_points'], hparams['normalize_scale'])
  get_data_augmentation(train_dataset, hparams['data_augmentation'], hparams['normalize_scale'], hparams['flip_axis'], hparams['flip_probability'],
                        hparams['rotate_axis'], hparams['rotate_degrees'])
  best_acc, state_dict_root = fit(train_dataset, valid_dataset, hparams['num_classes'], hparams['k'], hparams['bs'],
                                  hparams['epochs'], hparams['lr'])

  test_inference = test(test_dataset, state_dict_root)

def get_model_name(model, normalize_scale, data_augmentation, dropout, optimizer, scheduler):
  model_name = model
  model_name += '_normalized' if normalize_scale else '_notNormalized'
  model_name += "_" + data_augmentation if data_augmentation is not None else '_noDataAugmentation'
  model_name += "_" + str(dropout) if dropout is not None else '_noDropout'
  model_name += "_" + optimizer
  model_name += "_" + scheduler if scheduler is not None else "_noScheduler"
  return model_name

In [ ]:
# Experiment 1
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = False
hparams['dropout'] = None
hparams['scheduler'] = None
hparams['data_augmentation'] = None
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()


In [ ]:
# Experiment 2
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = None
hparams['scheduler'] = None
hparams['data_augmentation'] = None
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 3
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = None
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 4
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100 
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 5
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'rotate'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 6
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 7
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'StepLR'
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 8
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'StepLR'
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 9
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'OneCycleLR'
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 10
hparams['optimizer'] = 'Adam'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'OneCycleLR'
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 11
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 12
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'StepLR'
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 13
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'OneCycleLR'
hparams['data_augmentation'] = 'flip_rotation'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 14
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = None
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 15
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'StepLR'
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()

In [ ]:
# Experiment 16
hparams['optimizer'] = 'SGD'
hparams['normalize_scale'] = True
hparams['dropout'] = 0.3
hparams['scheduler'] = 'OneCycleLR'
hparams['data_augmentation'] = 'flip'
hparams['epochs'] = 100
hparams['model_name'] = get_model_name('pointNet', hparams['normalize_scale'], hparams['data_augmentation'], hparams['dropout'], hparams['optimizer'], hparams['scheduler'])
execute_experiment()